## Скоринг Авто Кредиты - учебный пример

Делаем скоринг. Удалил все малоинформативные и не до конца понятные мне столбцы. Не стал обращать внимание на статусы кредитов (Одобрен, отказан и так далее). Так гораздо больше данных для обучения модели. Просрочку считаем как максимум между просрочкой ОД и %.

**Таргет** - плохих клиентов определил двумя способами. 
- Клиенты, которые вышли на просрочку 60+
- Клиенты, допустившие просрочку 3 месяца подряд

## Подключаем библиотеки

In [ ]:
# подгружаем все нужные пакеты
import pandas as pd
import numpy as np

# игнорируем warnings
import warnings
warnings.filterwarnings("ignore")

import seaborn as sns

import matplotlib
import matplotlib.pyplot as plt
import matplotlib.ticker
%matplotlib inline

# настройка внешнего вида графиков в seaborn
sns.set_context(
    "notebook", 
    font_scale = 1.5,       
    rc = { 
        "figure.figsize" : (30, 30), 
        "axes.titlesize" : 18 
    }
)

## Загружаем данные

In [ ]:
data = pd.read_csv('auto_app.csv', delimiter=";",decimal=".", encoding="windows-1251")

## Первичный анализ данных

In [ ]:
data.shape

Посмотрим как признаки коррелируют между собой

In [ ]:
plt.figure(figsize = (50,50))
sns.heatmap(data.corr(),annot=True)

Все признаки с сильной корреляцией (свыше 0.6, менее -0.6 тоже) необходимо будет дополнительно проанализировать и удалить в случае необходимости.

Удалим ИИН. Порядковый номер тоже удалим.

In [ ]:
iin = data['IIN']
data = data.drop(('IIN'), axis=1)

In [ ]:
data = data.drop(data.columns[[0,1]], axis=1)

In [ ]:
data.head()

Выделим числовые и категориальные признаки:

In [ ]:
categorical_columns = [c for c in data.columns if data[c].dtype.name == 'object']
numerical_columns   = [c for c in data.columns if data[c].dtype.name != 'object']
print (categorical_columns)
print (numerical_columns)

In [ ]:
data[categorical_columns].describe()

In [ ]:
data[numerical_columns].describe()

## Пропущенные значения

In [ ]:
data.count(axis=0)

Всего у нас 17433. Попробуем разнные техники заполнения пустых значений. Возьмем для примера признак ПОЛ - заполним пропуски самым часто встречаемым.

In [ ]:
data['Пол'].describe()

In [ ]:
data['Пол'] = data['Пол'].fillna('Мужской')

In [ ]:
data[categorical_columns].describe()

Заполним для начала пропуски в определенных столбцах самыми часто встречаемыми значениями. Везде количество должно быть 17433. Если в столбце не хватает 40% или более данных, такой столбец можно удалять.

In [ ]:
(data[categorical_columns].count(axis=0)/data.shape[0])*100

In [ ]:
data = data.drop((['Статус занятости.1','Категория занимаемой должности','Тип платежа']), axis=1)

In [ ]:
data_describe = data.describe(include=[object])
for c in ['Гражданство','Резиденство','Образование','Семейное положение',\
          'Адрес фактического проживания совпадает с адресом регистрации?','Отношение к месту проживания', \
          'Филиал', 'СПФ','Канал продаж'
         ]:
    data[c] = data[c].fillna(data_describe[c]['top'])

Не трогаем поле **Агент**, из него мы сделаем позднее производное поле. Поле **Должность клиента** и **Вид деятельности компании/организации** потеряют очень много в информативности, если заменить часто встречаемым. Не могут все быть только Директорами. Поэтому удалим. Остальные заполним частовстречаемым.

In [ ]:
data = data.drop((['Должность клиента','Вид деятельности компании/организации']), axis=1)

In [ ]:
categorical_columns = [c for c in data.columns if data[c].dtype.name == 'object']
print (categorical_columns)

In [ ]:
(data[categorical_columns].count(axis=0)/data.shape[0])*100

Посмотрим на остальные с пропущенными данными более детально.

In [ ]:
data[['Цель кредитования','Условия кредитования','Наименование атосалона',\
      'Схема финансирования','Вид страхования','Категория клиента','Статус занятости','Имеется работа по совместительству?',\
     'Являетесь ли вы лицом, освобожденным от уплаты обязательных пенсионных взносов в НПФ',\
     'Кредитная история в БВУ (автомат)','Кредитная история в БВУ (А0)']].describe()

In [ ]:
data_describe = data.describe(include=[object])
for c in ['Цель кредитования','Условия кредитования','Наименование атосалона',\
          'Схема финансирования','Вид страхования','Категория клиента',\
          'Статус занятости','Имеется работа по совместительству?',\
          'Являетесь ли вы лицом, освобожденным от уплаты обязательных пенсионных взносов в НПФ'
         ]:
    data[c] = data[c].fillna(data_describe[c]['top'])

Пока не трогаем поле **Кредитная история в БВУ**. Самое часто встречаемое там **Безупречная**. Плосмотрим какие еще значения там есть. 

In [ ]:
data['Кредитная история в БВУ (автомат)'].unique()

In [ ]:
data['Кредитная история в БВУ (А0)'].unique()

Не верно говорить, что у всех у кого пустые данные поля Кредитная история Безупречная. Будем счиатать, что она **Отсутствует**

In [ ]:
for c in ['Кредитная история в БВУ (автомат)', 'Кредитная история в БВУ (А0)']:
    data[c] = data[c].fillna('Отсутствует')

In [ ]:
(data[categorical_columns].count(axis=0)/data.shape[0])*100

Посмотрим еще раз на данные

In [ ]:
categorical_columns = [c for c in data.columns if data[c].dtype.name == 'object']
print (categorical_columns)

In [ ]:
data[categorical_columns].count(axis=0)

In [ ]:
data[numerical_columns].count(axis=0)

Только в поле **Агент** остались пропуски. Займемся этим позднее.

## Векторизация 

Вначале выделим бинарные и небинарные признаки:

In [ ]:
binary_columns    = [c for c in categorical_columns if data_describe[c]['unique'] == 2]
nonbinary_columns = [c for c in categorical_columns if data_describe[c]['unique'] > 2]
print (binary_columns, nonbinary_columns)

### Бинарные признаки

Значения бинарных признаков просто заменим на 0 и 1.

In [ ]:
for c in binary_columns[0:]:
    top = data_describe[c]['top']
    top_items = data[c] == top
    data.loc[top_items, c] = 1
    data.loc[np.logical_not(top_items), c] = 0

In [ ]:
data[binary_columns].describe()

### Небинарные признаки

In [ ]:
nonbinary_columns 

In [ ]:
data[nonbinary_columns].describe()

Поля: **Кредитная история в БВУ (А0), Кредитная история в БВУ (автомат), Статус занятости, Категория клиента,Цель кредитования,Семейное положение** мы будем кодировать методом векторизации, который заключается в следующем.

Признак j, принимающий s значений, заменим на s признаков, принимащих значения 0 или 1, в зависимости от того, чему равно значение исходного признака j.

Например, признак A4 принимает 3 различных значения 'u', 'y', 'l'
    
Заменим признак A4 тремя признаками: A4_u, A4_y, A4_l.

- Если признак A4 принимает значение u, то признак A4_u равен 1, A4_y равен 0, A4_l равен 0.
- Если признак A4 принимает значение y, то признак A4_y равен 0, A4_y равен 1, A4_l равен 0.
- Если признак A4 принимает значение l, то признак A4_l равен 0, A4_y равен 0, A4_l равен 1.

К остальным мы этот метод не применяем, так как они содержат слишком много уникальных значения, для них примененим другой метод, что бы не потерять информативность

In [ ]:
data_dummies = pd.get_dummies(data[['Кредитная история в БВУ (А0)',\
                                   'Кредитная история в БВУ (автомат)', 'Статус занятости',\
                                   'Категория клиента',\
                                   'Цель кредитования','Семейное положение']])
print(data_dummies.columns)

Итак, остались признаки:'Гражданство', 'Образование', 'Отношение к месту проживания', 'Филиал', 'СПФ', 'Канал продаж',
 'Агент', 'Условия кредитования', 'Наименование атосалона'. Признак 'Агент' все так же не трогаем. В остальных признаках много значений, будем их кодировтаь средним.
Закодировав средними мы не потеряем информативность. Мы будем кодировать все признаки кроме 'Агент' средним от **target** таким образом мы поймем, например, по какому филиалу больше с среднем клиентов с просрочкой.

Можно это делать так же по другим параметрам. Наример по **Запрошенной сумме кредита**, тогда поймем где больше у нас запрашивают кредитов в среднем.

In [ ]:
data['Гражданство_mean'] = data.groupby(['Гражданство'])['target'].transform('mean')

In [ ]:
data['Образование_mean'] = data.groupby(['Образование'])['target'].transform('mean')

In [ ]:
data['Отношение к месту проживания_mean'] = data.groupby(['Отношение к месту проживания'])['target'].transform('mean')

In [ ]:
data['Филиал_mean'] = data.groupby(['Филиал'])['target'].transform('mean')

In [ ]:
data['СПФ_mean'] = data.groupby(['СПФ'])['target'].transform('mean')

In [ ]:
data['Канал продаж_mean'] = data.groupby(['Канал продаж'])['target'].transform('mean')

In [ ]:
data['Условия кредитования_mean'] = data.groupby(['Условия кредитования'])['target'].transform('mean')

In [ ]:
data['Наименование атосалона_mean'] = data.groupby(['Наименование атосалона'])['target'].transform('mean')

In [ ]:
data = data.drop((['Гражданство', 'Образование', 'Отношение к месту проживания', 'Филиал', 'СПФ', 'Канал продаж',\
                   'Условия кредитования', 'Наименование атосалона']), axis=1)

In [ ]:
data.head()

Признак Агент закодируем следующим образом, если оно не пустое напишем в новом поле 1, если пустое 0. Таким образом мы будем знать, что это поле было либо пустое, либо там был какой то агент. Отсутствие информации - тоже информация.

In [ ]:
data['Агент'].describe()

In [ ]:
data.loc[data['Агент'].notnull(), 'Агент_new'] = 1

In [ ]:
data.loc[data['Агент'].isnull(), 'Агент_new'] = 0

In [ ]:
data['Агент_new'].describe()

In [ ]:
data['Агент_new'].value_counts()

### Соединяем все в одну таблицу

In [ ]:
binary_columns

In [ ]:
data_dummies.columns

In [ ]:
numerical_columns   = [c for c in data.columns if data[c].dtype.name != 'object']

In [ ]:
print(numerical_columns)

In [ ]:
data_numerical = data[numerical_columns]

In [ ]:
data = pd.concat((data_numerical, data_dummies, data[binary_columns]), axis=1)
data = pd.DataFrame(data, dtype=float)
print (data.shape, data.columns)

Мы сделали очень много разных манипуляций с нашими данными. Полезно снова взглянуть на корреляцию. Графически это выгледело не слишком привлекательно и понятно. Поэтому взглянем в виде текста. Вывожу первых 60 пар.

In [ ]:
def get_redundant_pairs(df):
    pairs_to_drop = set()
    cols = df.columns
    for i in range(0, df.shape[1]):
        for j in range(0, i+1):
            pairs_to_drop.add((cols[i], cols[j]))
    return pairs_to_drop

def get_top_abs_correlations(df, n=5):
    au_corr = df.corr().abs().unstack()
    labels_to_drop = get_redundant_pairs(df)
    au_corr = au_corr.drop(labels=labels_to_drop).sort_values(ascending=False)
    return au_corr[0:n]

print("Наибольшая корреляция:")
print(get_top_abs_correlations(data, 60))

Мы видим, что очень многие признаки коррелирует очень сильно дург с другом и корреляция в ряде случаев почти равна 1. Это говорит о том, что либо признаки одинаковы либо очень линейно зависимы. Я удалю те, которые справа и выше 0.7 (В бою я б конечно обсудил бы это с экспертом). Если посмотреть внимательно, присутсвует и мультиколлениарность, то есть зависимость одного признака от нескольких.

In [ ]:
data = data.drop((['Коэффициент О/Д (автомат)', 'СПФ_mean', 'ВСЕГО совокупный расход', 'Статус занятости_Трудовой договор без срока (постоянная занятость)'\
                   ,'Кредитная история в БВУ (автомат)_Безупречная', 'Кредитная история в БВУ (автомат)_Положительная',\
                    'Всего совокупный доход Заемщика и Созаемщика','Комиссиия за рассмотрение кредитной заявки',\
                    'Оценочная стоимость по данным НОК', 'По основному месту работы/доходы от основной предпринимательской деятельности',\
                    'Первоначальный взнос','Категория клиента_Зарплатный проект',\
                   'Цель кредитования_Приобретение нового автотранспорта в автосалоне-партнере (государственная программа по поддержке отечественных автопроизводителей)'\
                   ,'Семейное положение_холост/незамужем','Комиссия за зачисление средств на счета физических лиц',\
                    'Кредитная история в БВУ (А0)_Отсутствует','Всего совокупный доход Заемщика и Созаемщика','Среднемесячный доход, подтвержденный ГЦВП (А0)',\
                    'Коэффициент О/Д (А0)','Еж.платежи в БВУ (А0)','Сумма ежемесячного платежа','Метод принятия решения','Переплата по кредиту',\
                    'Всего совокупный доход Заемщика и Созаемщика','Кредитная история в БВУ (автомат)_Отсутствует',\
                    'Процентная ставка']), axis=1)

In [ ]:
data.shape

Далее мы создадим отдельный вектор y с выходной (целевой) признак - ответом и удалим его из таблицы с данными для обучения


In [ ]:
y = data['target']
data = data.drop(('target'), axis=1)

### Обучающая и тестовая выборки

In [ ]:
from sklearn.model_selection import train_test_split

In [ ]:
data = data.drop('creditNumber', axis=1)

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(data, y, test_size = 0.3, random_state = 11)

In [ ]:
N_train, _ = X_train.shape 
N_test,  _ = X_test.shape 
print (N_train, N_test)

In [ ]:
print (y_train.shape, y_test.shape )

In [ ]:
y_train.value_counts()

In [ ]:
y_test.value_counts()

In [ ]:
(24/12179)*100

In [ ]:
(17/5213)*100

In [ ]:
((17+24)/(12179+5213))*100

Видно, что выборка несбалансированная - 0.23% плохих кредитов. Этого мало для качественного обучения модели. В таких ситуациях можно поступить двумя вариантам. Первый - увеличить количество плохих в выборке. Второй уменьшитть количество хороших. Этот метод называет АндерСэмплирование. Производится случайным образом. Сделаем, что бы доля стала хотя бы 10%.

In [ ]:
from collections import Counter
from imblearn.under_sampling import RandomUnderSampler

In [ ]:
ratio = {0: 369, 1: 41}
rus = RandomUnderSampler(random_state=42,ratio=ratio)
X_res, y_res = rus.fit_sample(data, y)
print('Размер нового датасета %s' % Counter(y_res))

In [ ]:
def plot_pie(y):
    target_stats = Counter(y)
    labels = list(target_stats.keys())
    sizes = list(target_stats.values())
    explode = tuple([0.1] * len(target_stats))

    fig, ax = plt.subplots()
    ax.pie(sizes, explode=explode, labels=labels, shadow=True,
           autopct='%1.1f%%')
    ax.axis('equal')
plot_pie(y_res)

plt.show()

In [ ]:
type(X_res)

Преобразуем numpy.ndarray в DataFrame и в Series

In [ ]:
df_n = pd.DataFrame(X_res, columns=list(data.columns.values))

In [ ]:
type(y_res)

In [ ]:
df_y = pd.DataFrame(y_res,columns=['target'])

In [ ]:
y_Series = df_y.ix[:,0]

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(df_n, y_Series, test_size = 0.3, random_state = 11)

In [ ]:
N_train, _ = X_train.shape 
N_test,  _ = X_test.shape 
print (N_train, N_test)

In [ ]:
print (y_train.shape, y_test.shape )

In [ ]:
y_train.value_counts()

In [ ]:
y_test.value_counts()

In [ ]:
(28/287)*100

In [ ]:
(13/123)*100

In [ ]:
((30+11)/(287+123))*100

## Строим модели

### Random Forest – случайный леc

Используем GridSearchCV для подбора лучших параметров модели

In [ ]:
from sklearn.model_selection import GridSearchCV
from sklearn.ensemble import RandomForestClassifier

rf = RandomForestClassifier(max_features='auto', oob_score=True, random_state=1, n_jobs=-1)

param_grid = { "criterion" : ["gini", "entropy"], "min_samples_leaf" : [1, 5, 10], "min_samples_split" : [2, 4, 10, 12, 16], "n_estimators": [50, 100, 400, 700, 1000]}

gs = GridSearchCV(estimator=rf, param_grid=param_grid, scoring='accuracy', cv=3, n_jobs=-1)

gs = gs.fit(X_train, y_train)

print(gs.best_params_)

In [ ]:
from sklearn.ensemble import RandomForestClassifier

rf = RandomForestClassifier(criterion='entropy', 
                             n_estimators=50,
                             min_samples_split=2,
                             min_samples_leaf=1,
                             max_features='auto',
                             oob_score=True,
                             random_state=1,
                             n_jobs=-1)
rf.fit(X_train, y_train)
print("%.4f" % rf.oob_score_)

In [ ]:
err_train = np.mean(y_train != rf.predict(X_train))
err_test  = np.mean(y_test  != rf.predict(X_test))
print (err_train, err_test)

Посмторим на значимые признаки.

In [ ]:
pd.concat((pd.DataFrame(data.columns, columns = ['variable']), 
           pd.DataFrame(rf.feature_importances_, columns = ['importance'])), 
          axis = 1).sort_values(by='importance', ascending = False)[:60]

In [ ]:
importances = rf.feature_importances_
indices = np.argsort(importances)[::-1]
feature_names = X_train.columns
d_first = 40
plt.figure(figsize=(30, 30))
plt.title("Важные признаки для определения просрочника")
plt.bar(range(d_first), importances[indices[:d_first]], align='center')
plt.xticks(range(d_first), np.array(feature_names)[indices[:d_first]], rotation=90)
plt.xlim([-1, d_first]);

Точность и полнота

Точность (precision) и полнота (recall) являются метриками которые используются при оценке большей части алгоритмов извлечения информации. Иногда они используются сами по себе, иногда в качестве базиса для производных метрик, таких как F-мера или R-Precision. Суть точности и полноты очень проста.

Точность системы в пределах класса – это доля документов действительно принадлежащих данному классу относительно всех документов которые система отнесла к этому классу. Полнота системы – это доля найденных классфикатором документов принадлежащих классу относительно всех документов этого класса в тестовой выборке.

Эти значения легко рассчитать на основании таблицы контингентности, которая составляется для каждого класса отдельно.
В таблице содержится информация сколько раз система приняла верное и сколько раз неверное решение по документам заданного класса. А именно:

TP — истино-положительное решение;
TN — истино-отрицательное решение;
FP — ложно-положительное решение;
FN — ложно-отрицательное решение.

Тогда, точность и полнота определяются следующим образом:
Precision=TP/(TP+FP)
Recall=TP/(TP+FN)

> Рассмотрим пример. Допустим, у вас есть тестовая выборка в которой 10 сообщений, из них 4 – спам. Обработав все сообщения классификатор пометил 2 сообщения как спам, причем одно действительно является спамом, а второе было помечено в тестовой выборке как нормальное. Мы имеем одно истино-положительное решение, три ложно-отрицательных и одно ложно-положительное. Тогда для класса “спам” точность классификатора составляет 12 (50% положительных решений правильные), а полнота 14 (классификатор нашел 25% всех спам-сообщений).

In [ ]:
from sklearn import metrics
a = rf.predict(X_train)
print(metrics.confusion_matrix(y_train, a))

In [ ]:
from sklearn.metrics import auc, accuracy_score, roc_auc_score

In [ ]:
print('The accuracy of prediction is:', accuracy_score(y_train, a))
print('The roc_auc_score of prediction is:', roc_auc_score(y_train, a))
print('The null acccuracy is:', max(y_test.mean(), 1 - y_train.mean()))

In [ ]:
from sklearn import metrics
a = rf.predict(X_test)
print(metrics.confusion_matrix(y_test, a))

In [ ]:
print('The accuracy of prediction is:', accuracy_score(y_test, a))
print('The roc_auc_score of prediction is:', roc_auc_score(y_test, a))
print('The null acccuracy is:', max(y_test.mean(), 1 - y_test.mean()))

In [ ]:
from sklearn import metrics
a = rf.predict(data)
print(metrics.confusion_matrix(y, a))

In [ ]:
print('The accuracy of prediction is:', accuracy_score(y, a))
print('The roc_auc_score of prediction is:', roc_auc_score(y, a))
print('The null acccuracy is:', max(y_test.mean(), 1 - y.mean()))

In [ ]:
y_pred_prob = rf.predict_proba(data)[:, 1]

In [ ]:
fpr, tpr, thresholds = metrics.roc_curve(y, y_pred_prob)

plt.plot(fpr, tpr)
plt.xlim([0.0, 1.0])
plt.ylim([0.0, 1.0])
plt.rcParams['font.size'] = 12
plt.title('ROC curve')
plt.xlabel('False Positive Rate (1 - Specificity)')
plt.grid(True)

In [ ]:
print(metrics.roc_auc_score(y, y_pred_prob))

In [ ]:
y_pred_prob

In [ ]:
a = rf.predict(data)
pd.DataFrame(a).to_csv('prediction.csv')
print(a)

In [ ]:
type(y_pred_prob)

### Градиентный бустинг - lightGBM

In [ ]:
import lightgbm as lgb
from sklearn.model_selection import GridSearchCV

In [ ]:
estimator = lgb.LGBMClassifier(learning_rate = 0.125, metric = 'l1', 
                        n_estimators = 20, num_leaves = 38)

param_grid = {
    'n_estimators': [x for x in range(50, 1050,50)],
    'learning_rate': [ 0.01, 0.005,0.10, 0.125, 0.15, 0.175, 0.2]}
gridsearch = GridSearchCV(estimator, param_grid)

gridsearch.fit(X_train, y_train,
        eval_set=[(X_test, y_test)],
        eval_metric=['auc', 'binary_logloss'],
early_stopping_rounds=5)

In [ ]:
print('Best parameters found by grid search are:', gridsearch.best_params_)

In [ ]:
gbm = lgb.LGBMClassifier(learning_rate = 0.175, metric = 'l1', 
                        n_estimators = 50, num_leaves = 38)
gbm.fit(X_train, y_train,
        eval_set=[(X_test, y_test)],
        eval_metric=['auc', 'binary_logloss'],
early_stopping_rounds=5)

In [ ]:
from sklearn.metrics import auc, accuracy_score, roc_auc_score

In [ ]:
y_pred = gbm.predict(X_test, num_iteration=gbm.best_iteration_)
print('The accuracy of prediction is:', accuracy_score(y_test, y_pred))
print('The roc_auc_score of prediction is:', roc_auc_score(y_test, y_pred))
print('The null acccuracy is:', max(y_test.mean(), 1 - y_test.mean()))

In [ ]:
from sklearn import metrics
print(metrics.confusion_matrix(y_pred, y_test))

In [ ]:
ax = lgb.plot_importance(gbm, height = 0.4, max_num_features=25, xlim = (0,100), ylim = (0,23), 
                         figsize = (10,6))
plt.show()